In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os

## Data Augmentation
- Rotation/reflection: 隨機旋轉圖像一定角度;改變圖像內容方向
- flip: 沿著水平或垂直方向翻轉圖像
- zoom: 按照一定的比例放大或縮小圖像
- shift: 在圖像平面上對圖像以一定方式進行平移(隨機或人為指定平移範圍和平移方向)
- scale: 對圖像按照指定的scale進行放大或縮小;或者利用指定的scale改變圖像的模糊程度
- contrast: 在圖像的HSV空間中,改變飽和度Ｓ和亮度Ｖ分量（對Ｓ跟Ｖ進行指數運算）,並保持色調不變
- noise: 對圖像的每個像素加上隨機的noise
- color: 在圖像空間中進行ＰＣＡ,得到ＲＧＢ空間中三個特徵向量跟特徵值,得到原ＲＧＢ像素空間的 representation.
- more: google it

## ImageDataGenerator
- Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely.
- You can use next(datagen) to get a batch of data

In [ ]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(nor_x_train)

In [ ]:
t_x, t_y = next(datagen.flow(nor_x_train, v_y_train, batch_size = 5))
plt.imshow(t_x[3]) # index form 0 ~ 4
print("Label is {}.".format(t_y[3]))

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=nor_x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 3
num_predictions = 20

model.fit_generator(datagen.flow(nor_x_train, v_y_train,
                                     batch_size=batch_size),
                        steps_per_epoch= x_train.shape[0] // batch_size,   
                        epochs=epochs,
                        validation_data=(nor_x_val, v_y_val),
                        workers=4)

In [ ]:
score = model.evaluate(nor_x_test, v_y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
pred_index = 15
pred = model.predict_on_batch(nor_x_test[pred_index].reshape(-1, 32, 32, 3))
plt.imshow(x_test[pred_index], cmap="gray")
print("Label is {}.".format(np.argmax(pred)))